In [3]:
# coding: utf-8
import numpy as np
import pandas as pd
# наше всё

pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)
#настройки, с которыми лучше

In [4]:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1

# get json form site 
def get_bch(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
    
    return st

# and write to folder
def write_bch(player, year, month):
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    for cs in range(3):
        try:
            st=get_bch(player, year, month)
            file = open('source/'+player+'_'+year+'_'+month+'_'+now+'.json','w') 
            file.write(st)
            file.close()
        except Exception:
            time.sleep(cs)
            error_st=response.status_code+st[0:3000]
            file = open('source/'+player+'_'+year+'_'+month+'_'+now+'.json','w') 
            file.write( error_st)
            file.close()

# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('source/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('source/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()

In [5]:
# rul=pd.read_json('source/players_list_RU_2019-07-28 23:48:08.json', orient='records')

In [6]:
# len(rul)

In [7]:
# rul[0:3]

In [8]:
user_list=['rosolimo']
#user_list=rul['players'].values[0:20]

In [9]:
user_list

['rosolimo']

In [8]:
# start getting with threading magic
import threading
i=0
for user in user_list:
    for year in range(2001, 2020):
        for month in range(1,13):
            t = threading.Thread(target=write_bch, args=(user, str(year), get_number(month)))
            t.start()
            i=i+1
print(i)

228


In [53]:
def get_data_from_file(path):
    total_info=pd.read_json(path)
    
    # number of games
    num=len(total_info)

    game_stat=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(total_info.values[i])
        game_stat=pd.concat([game_stat, b])
        
    return game_stat

# df=get_data_from_file('source/rosolimo_2018_04_2019-08-04 22:21:13.json')

In [54]:
# чистаем все json, что лежат в папке тест
l=[]
import os
for root, dirs, files in os.walk("source/"):
    for file in files:
        if file.endswith(".json"):
             l.append(os.path.join(root, file))
                   
# и пишем в pandas
i=0
df=pd.DataFrame()
for link in l:
    try:
        dfb=get_data_from_file(link)
    except:
        dfb=pd.DataFrame()
    df=pd.concat([df, dfb])

In [55]:
len(df)

1060

In [59]:
df[0:2]

,black.@id,black.rating,black.result,black.username,end_time,fen,pgn,rated,rules,time_class,time_control,url,white.@id,white.rating,white.result,white.username
0,https://api.chess.com/pub/player/rosolimo,1661,win,Rosolimo,1519903019,5r2/ppk3p1/2p1p1p1/2n1N1P1/4P3/P7/2P2qB1/3RR1K1 w - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Date ""2018.03.01""]\n[Round ""-""]\n[White ""Mltr-m3awia""]\n[Black ""Rosolimo""]\n[Result ""0-1""]\n[ECO ""A07""]\n[ECOUrl ""https://www.chess.com/openings/A07-Kings-Indian-Attack-Yugoslav-Variation-3.Bg2-c6""]\n[CurrentPosition ""5r2/ppk3p1/2p1p1p1/2n1N1P1/4P3/P7/2P2qB1/3RR1K1 w - -""]\n[Timezone ""UTC""]\n[UTCDate ""2018.03.01""]\n[UTCTime ""11:10:29""]\n[WhiteElo ""1625""]\n[BlackElo ""1661""]\n[TimeControl ""180+2""]\n[Termination ""Rosolimo won on time""]\n[StartTime ""11:10:29""]\n[EndDate ""2018.03.01""]\n[EndTime ""11:16:59""]\n[Link ""https://www.chess.com/live/game/2648272097""]\n\n1. Nf3 {[%clk 0:03:01.7]} 1... d5 {[%clk 0:03:00.7]} 2. g3 {[%clk 0:03:01.2]} 2... c6 {[%clk 0:03:01.5]} 3. Bg2 {[%clk 0:02:59.2]} 3... Nf6 {[%clk 0:03:02.2]} 4. O-O {[%clk 0:03:00]} 4... Bf5 {[%clk 0:03:01.9]} 5. d3 {[%clk 0:03:00.5]} 5... e6 {[%clk 0:02:59.2]} 6. Nbd2 {[%clk 0:03:01.2]} 6... Bd6 {[%clk 0:02:59.1]} 7. Nh4 {[%clk 0:03:01.7]} 7... Bg6 {[%clk 0:02:59.3]} 8. e4 {[%clk 0:02:59]} 8... dxe4 {[%clk 0:02:59.7]} 9. dxe4 {[%clk 0:03:00]} 9... Nbd7 {[%clk 0:02:48.3]} 10. Nxg6 {[%clk 0:02:59.2]} 10... hxg6 {[%clk 0:02:48.8]} 11. Nc4 {[%clk 0:02:57.2]} 11... Bc5 {[%clk 0:02:43.5]} 12. Bg5 {[%clk 0:02:55.3]} 12... Qc7 {[%clk 0:02:43.1]} 13. a3 {[%clk 0:02:26.9]} 13... O-O-O {[%clk 0:02:41.4]} 14. Qe2 {[%clk 0:02:27.2]} 14... Rh5 {[%clk 0:02:41.5]} 15. h4 {[%clk 0:01:56.3]} 15... Qxg3 {[%clk 0:02:35.5]} 16. b4 {[%clk 0:01:46.5]} 16... Ng4 {[%clk 0:02:21.7]} 17. Qxg4 {[%clk 0:01:14.4]} 17... Qxg4 {[%clk 0:01:41.7]} 18. Rad1 {[%clk 0:01:00.1]} 18... Rxg5 {[%clk 0:01:26.5]} 19. hxg5 {[%clk 0:00:56.4]} 19... Rh8 {[%clk 0:01:26.1]} 20. bxc5 {[%clk 0:00:49.4]} 20... Qh4 {[%clk 0:01:26.8]} 21. Rfe1 {[%clk 0:00:42.2]} 21... Nxc5 {[%clk 0:01:24]} 22. Nd6+ {[%clk 0:00:42.5]} 22... Kc7 {[%clk 0:01:22.3]} 23. Nxf7 {[%clk 0:00:42.5]} 23... Rf8 {[%clk 0:01:23.2]} 24. Ne5 {[%clk 0:00:07.6]} 24... Qxf2+ {[%clk 0:01:18.4]} 0-1",True,chess,blitz,180+2,https://www.chess.com/live/game/2648272097,https://api.chess.com/pub/player/mltr-m3awia,1625,timeout,Mltr-m3awia
0,https://api.chess.com/pub/player/vzheka,1646,win,vzheka,1519977143,6k1/1R3p2/2B3p1/qQ2p2p/P5nP/4P1PK/7r/8 w - -,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Date ""2018.03.02""]\n[Round ""-""]\n[White ""Rosolimo""]\n[Black ""vzheka""]\n[Result ""0-1""]\n[ECO ""A41""]\n[ECOUrl ""https://www.chess.com/openings/A41-Modern-Defense""]\n[CurrentPosition ""6k1/1R3p2/2B3p1/qQ2p2p/P5nP/4P1PK/7r/8 w - -""]\n[Timezone ""UTC""]\n[UTCDate ""2018.03.02""]\n[UTCTime ""07:44:23""]\n[WhiteElo ""1652""]\n[BlackElo ""1646""]\n[TimeControl ""180+2""]\n[Termination ""vzheka won by checkmate""]\n[StartTime ""07:44:23""]\n[EndDate ""2018.03.02""]\n[EndTime ""07:52:23""]\n[Link ""https://www.chess.com/live/game/2650198907""]\n\n1. d4 {[%clk 0:03:01.8]} 1... g6 {[%clk 0:03:00.6]} 2. c4 {[%clk 0:03:02.7]} 2... d6 {[%clk 0:03:01.8]} 3. Nc3 {[%clk 0:03:03.2]} 3... Bg7 {[%clk 0:03:03.3]} 4. Nf3 {[%clk 0:03:02.5]} 4... Nc6 {[%clk 0:03:04.7]} 5. e3 {[%clk 0:03:00.8]} 5... Bg4 {[%clk 0:03:05.6]} 6. Be2 {[%clk 0:03:01.6]} 6... e5 {[%clk 0:03:06.3]} 7. dxe5 {[%clk 0:03:01.8]} 7... Bxf3 {[%clk 0:02:56.9]} 8. Bxf3 {[%clk 0:02:58.3]} 8... Bxe5 {[%clk 0:02:53.9]} 9. O-O {[%clk 0:02:58]} 9... Qh4 {[%clk 0:02:50.4]} 10. g3 {[%clk 0:02:57.7]} 10... Qf6 {[%clk 0:02:32.2]} 11. Bg2 {[%clk 0:02:45.6]} 11... Bxc3 {[%clk 0:02:25.4]} 12. bxc3 {[%clk 0:02:46.4]} 12... Qxc3 {[%clk 0:02:26.2]} 13. Rb1 {[%clk 0:02:47.3]} 13... Qxc4 {[%clk 0:02:17.5]} 14. Rxb7 {[%clk 0:02:42.5]} 14... Rc8 {[%clk 0:02:01.3]} 15. Bb2 {[%clk 0:02:41.2]} 15... Ne5 {[%clk 0:01:54]} 16. Bd5 {[%clk 0:02:24.1]} 16... Qa6 {[%clk 0:01:35.5]

In [76]:
# unixtime to time
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# working with pgn
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    
    # number of games
    num=len(game_stat)
    
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)

    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 
        'result', 'date', 'time', 
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'

    ]]
    
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
    
    return game_stat


def find_col(col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
    
def find_opp(col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val
    
def learn_prepair(df, target_user):
    df['target_user']=target_user
    df['score']=df[['white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    return df
    

In [77]:
dfs=get_pl_stat(df)

In [74]:
dfs[0:2]

,rules,time_class,time_control,rated,result,date,time,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,eco,eco_url,game_url,white_url,black_url
0,chess,blitz,180+2,True,0-1,2018.03.01,11:10:29,Mltr-m3awia,Rosolimo,1625,1661,timeout,win,0.0,1.0,A07,https://www.chess.com/openings/A07-Kings-Indian-Attack-Yugoslav-Variation-3.Bg2-c6,https://www.chess.com/live/game/2648272097,https://api.chess.com/pub/player/mltr-m3awia,https://api.chess.com/pub/player/rosolimo
0,chess,blitz,180+2,True,0-1,2018.03.02,07:44:23,Rosolimo,vzheka,1652,1646,checkmated,win,0.0,1.0,A41,https://www.chess.com/openings/A41-Modern-Defense,https://www.chess.com/live/game/2650198907,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/vzheka


In [75]:
len(dfs)

1060